################################################################################
> # **Part - I**

*   define actor critic networks
*   define PPO algorithm

################################################################################

In [1]:


############################### Import libraries ###############################


import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

import gym
import pongGym

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")




################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

#         self.quant = torch.quantization.QuantStub()
        self.actor = nn.Sequential(
                        nn.Linear(state_dim, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, action_dim),
                        nn.Softmax(dim=-1)
                    )

#         self.dequant = torch.quantization.DeQuantStub()
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 1)
                    )

    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        
        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action):


        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def select_action(self, state):

        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob = self.policy_old.act(state)

        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logprob)

        return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        
        
       



pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Device set to : NVIDIA GeForce RTX 3070


################################################################################
> # **Part - II**

*   train PPO algorithm on environments
*   save preTrained networks weights and log files

################################################################################

In [ ]:

print("============================================================================================")


################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "pong_game"

max_ep_len = 60*60                    # max timesteps in one episode
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyperparameters ################


update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 60*3               # update policy for K epochs
# K_epochs = 4
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)

#####################################################



print("training environment name : " + env_name)

env = pongGym.DoublePong()

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension

action_dim = env.action_space.n



###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################


################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps") 
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0

Max_reward = 0
# training loop
# env = pongGym.DoublePong()
while i_episode < 100000:
    
    state = env.reset()
    current_ep_reward = 0
    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
#         if i_episode > 100:
#             env.render()
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0
            
        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))
            print_running_reward = 0
            print_running_episodes = 0
        # break; if the episode is over
        if done:
            break

    # save model weights
#     print("Episode : {} \t\t Totol reward : {}".format(i_episode, current_ep_reward))
    if current_ep_reward > Max_reward:
        Max_reward = current_ep_reward
        checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, int(Max_reward), run_num_pretrained)
        print("--------------------------------------------------------------------------------------------")
        print("saving model at : " + checkpoint_path)
        ppo_agent.save(checkpoint_path)
        print("model saved")
        print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
        print("--------------------------------------------------------------------------------------------")
    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")







training environment name : pong_game
current logging run number for pong_game :  0
logging at : PPO_logs/pong_game//PPO_pong_game_log_0.csv
--------------------------------------------------------------------------------------------
max training timesteps :  100000
max timesteps per episode :  3600
model saving frequency : 20000 timesteps
log frequency : 7200 timesteps
printing average reward over episodes in last : 14400 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  22
action space dimension :  4
--------------------------------------------------------------------------------------------
Initializing a discrete action space policy
--------------------------------------------------------------------------------------------
PPO update frequency : 14400 timesteps
PPO K epochs :  180
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
---------------------------------------------------------------------

Episode : 379 		 Timestep : 1368000 		 Average Reward : -169.58
Episode : 383 		 Timestep : 1382400 		 Average Reward : -194.15
Episode : 387 		 Timestep : 1396800 		 Average Reward : -205.69
Episode : 391 		 Timestep : 1411200 		 Average Reward : -184.84
Episode : 395 		 Timestep : 1425600 		 Average Reward : -185.26
Episode : 399 		 Timestep : 1440000 		 Average Reward : -179.88
Episode : 403 		 Timestep : 1454400 		 Average Reward : -237.32
Episode : 407 		 Timestep : 1468800 		 Average Reward : -136.27
Episode : 411 		 Timestep : 1483200 		 Average Reward : -208.75
Episode : 415 		 Timestep : 1497600 		 Average Reward : -115.48
Episode : 419 		 Timestep : 1512000 		 Average Reward : -104.81
Episode : 423 		 Timestep : 1526400 		 Average Reward : -119.02
Episode : 427 		 Timestep : 1540800 		 Average Reward : -186.17
Episode : 431 		 Timestep : 1555200 		 Average Reward : -184.22
Episode : 435 		 Timestep : 1569600 		 Average Reward : -166.89
Episode : 439 		 Timestep : 1584000 		 A

Episode : 803 		 Timestep : 2894400 		 Average Reward : -91.21
Episode : 807 		 Timestep : 2908800 		 Average Reward : -190.46
Episode : 811 		 Timestep : 2923200 		 Average Reward : -171.97
Episode : 815 		 Timestep : 2937600 		 Average Reward : -127.55
Episode : 819 		 Timestep : 2952000 		 Average Reward : -234.22
Episode : 823 		 Timestep : 2966400 		 Average Reward : -92.07
Episode : 827 		 Timestep : 2980800 		 Average Reward : -74.27
Episode : 831 		 Timestep : 2995200 		 Average Reward : -257.17
Episode : 835 		 Timestep : 3009600 		 Average Reward : -159.95
Episode : 839 		 Timestep : 3024000 		 Average Reward : -47.47
Episode : 843 		 Timestep : 3038400 		 Average Reward : -107.66
Episode : 847 		 Timestep : 3052800 		 Average Reward : -91.12
Episode : 851 		 Timestep : 3067200 		 Average Reward : -138.56
Episode : 855 		 Timestep : 3081600 		 Average Reward : -122.53
Episode : 859 		 Timestep : 3096000 		 Average Reward : -56.66
Episode : 863 		 Timestep : 3110400 		 Average

Episode : 1299 		 Timestep : 4680000 		 Average Reward : -33.12
Episode : 1303 		 Timestep : 4694400 		 Average Reward : -142.77
Episode : 1307 		 Timestep : 4708800 		 Average Reward : -65.09
Episode : 1311 		 Timestep : 4723200 		 Average Reward : -141.3
Episode : 1315 		 Timestep : 4737600 		 Average Reward : -71.11
Episode : 1319 		 Timestep : 4752000 		 Average Reward : -176.12
Episode : 1323 		 Timestep : 4766400 		 Average Reward : -70.2
Episode : 1327 		 Timestep : 4780800 		 Average Reward : -134.83
Episode : 1331 		 Timestep : 4795200 		 Average Reward : -53.59
Episode : 1335 		 Timestep : 4809600 		 Average Reward : -86.98
Episode : 1339 		 Timestep : 4824000 		 Average Reward : -119.32
Episode : 1343 		 Timestep : 4838400 		 Average Reward : 38.36
Episode : 1347 		 Timestep : 4852800 		 Average Reward : -27.12
Episode : 1351 		 Timestep : 4867200 		 Average Reward : -110.27
Episode : 1355 		 Timestep : 4881600 		 Average Reward : -96.41
Episode : 1359 		 Timestep : 4896000 

Episode : 1795 		 Timestep : 6465600 		 Average Reward : -39.91
Episode : 1799 		 Timestep : 6480000 		 Average Reward : -154.24
Episode : 1803 		 Timestep : 6494400 		 Average Reward : -67.54
Episode : 1807 		 Timestep : 6508800 		 Average Reward : -115.21
Episode : 1811 		 Timestep : 6523200 		 Average Reward : -108.9
Episode : 1815 		 Timestep : 6537600 		 Average Reward : -76.14
Episode : 1819 		 Timestep : 6552000 		 Average Reward : -75.8
Episode : 1823 		 Timestep : 6566400 		 Average Reward : -58.76
Episode : 1827 		 Timestep : 6580800 		 Average Reward : -43.19
Episode : 1831 		 Timestep : 6595200 		 Average Reward : -94.02
Episode : 1835 		 Timestep : 6609600 		 Average Reward : -149.19
Episode : 1839 		 Timestep : 6624000 		 Average Reward : -128.47
Episode : 1843 		 Timestep : 6638400 		 Average Reward : -150.41
Episode : 1847 		 Timestep : 6652800 		 Average Reward : -160.85
Episode : 1851 		 Timestep : 6667200 		 Average Reward : -131.67
Episode : 1855 		 Timestep : 66816

...

Episode : 12879 		 Timestep : 46368000 		 Average Reward : -50.38
Episode : 12883 		 Timestep : 46382400 		 Average Reward : 80.11
Episode : 12887 		 Timestep : 46396800 		 Average Reward : -9.06
Episode : 12891 		 Timestep : 46411200 		 Average Reward : -44.0
Episode : 12895 		 Timestep : 46425600 		 Average Reward : 14.23
Episode : 12899 		 Timestep : 46440000 		 Average Reward : -43.33
Episode : 12903 		 Timestep : 46454400 		 Average Reward : 29.02
Episode : 12907 		 Timestep : 46468800 		 Average Reward : 19.97
Episode : 12911 		 Timestep : 46483200 		 Average Reward : 10.14
Episode : 12915 		 Timestep : 46497600 		 Average Reward : -11.88
Episode : 12919 		 Timestep : 46512000 		 Average Reward : 56.27
Episode : 12923 		 Timestep : 46526400 		 Average Reward : 29.44
Episode : 12927 		 Timestep : 46540800 		 Average Reward : 41.46
Episode : 12931 		 Timestep : 46555200 		 Average Reward : 39.83
Episode : 12935 		 Timestep : 46569600 		 Average Reward : -8.16
Episode : 12939 		 Tim

Episode : 13383 		 Timestep : 48182400 		 Average Reward : 76.1
Episode : 13387 		 Timestep : 48196800 		 Average Reward : 20.3
Episode : 13391 		 Timestep : 48211200 		 Average Reward : 49.84
Episode : 13395 		 Timestep : 48225600 		 Average Reward : 8.47
Episode : 13399 		 Timestep : 48240000 		 Average Reward : 6.36
Episode : 13403 		 Timestep : 48254400 		 Average Reward : 62.75
Episode : 13407 		 Timestep : 48268800 		 Average Reward : -33.14
Episode : 13411 		 Timestep : 48283200 		 Average Reward : 46.32
Episode : 13415 		 Timestep : 48297600 		 Average Reward : 43.61
Episode : 13419 		 Timestep : 48312000 		 Average Reward : -13.07
Episode : 13423 		 Timestep : 48326400 		 Average Reward : 13.65
Episode : 13427 		 Timestep : 48340800 		 Average Reward : 2.06
Episode : 13431 		 Timestep : 48355200 		 Average Reward : 8.04
Episode : 13435 		 Timestep : 48369600 		 Average Reward : 25.24
Episode : 13439 		 Timestep : 48384000 		 Average Reward : 24.15
Episode : 13443 		 Timestep :

Episode : 13891 		 Timestep : 50011200 		 Average Reward : 50.41
Episode : 13895 		 Timestep : 50025600 		 Average Reward : 98.47
Episode : 13899 		 Timestep : 50040000 		 Average Reward : 48.22
Episode : 13903 		 Timestep : 50054400 		 Average Reward : 45.23
Episode : 13907 		 Timestep : 50068800 		 Average Reward : 22.09
Episode : 13911 		 Timestep : 50083200 		 Average Reward : 16.18
Episode : 13915 		 Timestep : 50097600 		 Average Reward : 9.5
Episode : 13919 		 Timestep : 50112000 		 Average Reward : 58.7
Episode : 13923 		 Timestep : 50126400 		 Average Reward : 130.32
Episode : 13927 		 Timestep : 50140800 		 Average Reward : 18.37
Episode : 13931 		 Timestep : 50155200 		 Average Reward : 18.85
Episode : 13935 		 Timestep : 50169600 		 Average Reward : 30.47
Episode : 13939 		 Timestep : 50184000 		 Average Reward : 14.86
Episode : 13943 		 Timestep : 50198400 		 Average Reward : -24.2
Episode : 13947 		 Timestep : 50212800 		 Average Reward : 52.26
Episode : 13951 		 Timestep

Episode : 14399 		 Timestep : 51840000 		 Average Reward : 7.62
Episode : 14403 		 Timestep : 51854400 		 Average Reward : 33.84
Episode : 14407 		 Timestep : 51868800 		 Average Reward : -12.74
Episode : 14411 		 Timestep : 51883200 		 Average Reward : -8.61
Episode : 14415 		 Timestep : 51897600 		 Average Reward : 0.58
Episode : 14419 		 Timestep : 51912000 		 Average Reward : -27.04
Episode : 14423 		 Timestep : 51926400 		 Average Reward : 87.02
Episode : 14427 		 Timestep : 51940800 		 Average Reward : 26.14
Episode : 14431 		 Timestep : 51955200 		 Average Reward : 12.13
Episode : 14435 		 Timestep : 51969600 		 Average Reward : 54.48
Episode : 14439 		 Timestep : 51984000 		 Average Reward : 7.46
Episode : 14443 		 Timestep : 51998400 		 Average Reward : -6.35
Episode : 14447 		 Timestep : 52012800 		 Average Reward : -29.17
Episode : 14451 		 Timestep : 52027200 		 Average Reward : -16.97
Episode : 14455 		 Timestep : 52041600 		 Average Reward : 9.05
Episode : 14459 		 Timest

Episode : 14907 		 Timestep : 53668800 		 Average Reward : 38.2
Episode : 14911 		 Timestep : 53683200 		 Average Reward : 84.97
Episode : 14915 		 Timestep : 53697600 		 Average Reward : -76.63
Episode : 14919 		 Timestep : 53712000 		 Average Reward : 19.12
Episode : 14923 		 Timestep : 53726400 		 Average Reward : 46.25
Episode : 14927 		 Timestep : 53740800 		 Average Reward : 8.52
Episode : 14931 		 Timestep : 53755200 		 Average Reward : 118.78
Episode : 14935 		 Timestep : 53769600 		 Average Reward : 7.56
Episode : 14939 		 Timestep : 53784000 		 Average Reward : 14.34
Episode : 14943 		 Timestep : 53798400 		 Average Reward : -59.63
Episode : 14947 		 Timestep : 53812800 		 Average Reward : 40.71
Episode : 14951 		 Timestep : 53827200 		 Average Reward : -32.84
Episode : 14955 		 Timestep : 53841600 		 Average Reward : -8.0
Episode : 14959 		 Timestep : 53856000 		 Average Reward : 25.57
Episode : 14963 		 Timestep : 53870400 		 Average Reward : 65.07
Episode : 14967 		 Timest

In [ ]:
checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, "final", run_num_pretrained)
print("--------------------------------------------------------------------------------------------")
print("saving model at : " + checkpoint_path)
ppo_agent.save(checkpoint_path)
print("model saved")
print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
print("--------------------------------------------------------------------------------------------")